In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import cv2
from skimage.transform import resize
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Conv2D,Reshape,Dense,Conv2DTranspose,BatchNormalization,LeakyReLU,Add,Activation,Input
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
img_h = 128
img_w = 128
img_c = 3
d = img_h,img_w,img_c

In [ ]:
def discriminator_model(img_size):
    
    i = Input(shape=(128,128,3))
    
    x = Flatten()(i)

    x = Dense(1024,activation=LeakyReLU())(x)
    
    x = Dense(512,activation=LeakyReLU())(x)
    
    x = Dense(256,activation=LeakyReLU())(x)
    
    x = Dense(1,activation='sigmoid')(x)
    
    model = Model(i,x)
    return model


In [ ]:

latent_dim = 100

def generator_model():
    
    i = Input(shape=(latent_dim,))
    x = Dense(8*8*256, use_bias=False, input_shape=(100,))(i)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = Reshape((8, 8, 256))(x)

    x = Conv2DTranspose(128, (5, 5), strides=(2,2), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False,)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)
    
    model = Model(i,x)

    return model

    
    
    
#     i = Input(shape = latent_dim,)
#     x = Dense(256,activation=LeakyReLU())(i)
#     x = BatchNormalization()(x)
    
#     x = Dense(512,activation=LeakyReLU())(x)
#     x = BatchNormalization()(x)
    
#     x = Dense(1024,activation=LeakyReLU())(x)
#     x = BatchNormalization()(x)
    
# #     x = Conv2D(,kernel_size=1,activation='tanh')(x)
# #     x = Reshape((128,128,3))(x)
   
    
#     model = Model(i,x)

#     return model


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

In [ ]:
discriminator = discriminator_model(d)
discriminator.compile( loss='binary_crossentropy', optimizer= discriminator_optimizer , metrics=['accuracy'])

generator = generator_model()

In [ ]:
discriminator.summary()

In [ ]:
generator.summary()

In [ ]:
path = '../input/animals/images/butterfly'
list_img = sorted(os.listdir(path))

In [ ]:

img = []
for i in list_img:
    a = cv2.imread(path+'/'+i)
    a = cv2.cvtColor(a,cv2.COLOR_RGB2BGR)
    a = cv2.resize(a,(img_h,img_w))
    img.append(a)
    

img = np.array(img)
# img = img / 255.0 * 2 - 1
# img = img.reshape(-1,d)
print(img.shape)

# ***************************************************************************************
# img = []
# for i in list_img:
#     a = path+'/'+i
#     b = os.listdir(a)
#     for j in b:
#         fnl_pth = cv2.imread(a+'/'+j)
#         fnl_pth = cv2.cvtColor(fnl_pth,cv2.COLOR_RGB2BGR)
#         fnl_pth = cv2.resize(fnl_pth,(img_h,img_w))
#         img.append(fnl_pth)

# img = np.array(img)
# print(img.shape)

In [ ]:
plt.imshow(img[655])

In [ ]:
z = Input(shape=(latent_dim,))
print(z.shape)

gen = generator(z)
print(gen.shape)

discriminator.trainable = False

fake_pred = discriminator(gen)

combined_model = Model(z, fake_pred)

combined_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [ ]:
batch_size = 32
epochs = 10000 + 1
sample_period = 1000

In [ ]:
zeros = np.zeros(batch_size)
ones = np.ones(batch_size)

d_losses = []
g_losses = []

if not os.path.exists('gan_img'):
    os.makedirs('gan_img')

In [ ]:
def sample_images(epoch):
  rows, cols = 2, 2
  noise = np.random.randn(rows * cols, latent_dim)
  imgs = generator.predict(noise)

  # Rescale images 0 - 1
  imgs = 0.5 * imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(img_h, img_w,img_c), cmap='gray')
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("gan_img/%d.png" % epoch)
  plt.close()

In [ ]:
# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
#                                  discriminator_optimizer=discriminator_optimizer,
#                                  generator=generator,
#                                  discriminator=discriminator)

In [ ]:

for epoch in range(epochs):
      
    idx = np.random.randint(0, img.shape[0], batch_size)
    real_imgs = img[idx]
    
    
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)
    
    
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)

    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc  = 0.5 * (d_acc_real + d_acc_fake)
    
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)
    
    
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)
    
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    
    if epoch % 100 == 0:
      print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
        d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
    
    if epoch % sample_period == 0:
      sample_images(epoch)
    
#     if epoch  == (epochs/2):
#         generator.save('generator{}.h5'.format(epoch))
    
    
#     if epochs == epoch:
#         generator.save('generator{}.h5'.format(epoch))
#         combined_model.save('combined_model.h5')
#         discriminator.save('discriminator.h5')
    

In [ ]:
generator.save('generator.h5')

In [ ]:
generator()

In [ ]:
import numpy as np
from keras.models import load_model
from numpy import asarray
import matplotlib.pyplot as plt
from numpy.random import randn

In [ ]:
model_c = load_model('../input/generator/generator (1).h5')

In [ ]:
vector =  np.random.randn(1, 100) #Vector of random numbers (creates a column, need to reshape)
# vector = vector.reshape(1, 100)

# generate image
X = model_c.predict(vector)

# plot the result
plt.imshow(X[0],cmap='gray')

In [ ]:
# combined_model.save('cm.h5')

In [ ]:
a = cv2.imread('./gan_img/{}.png'.format(220000))
plt.imshow(a)